In [1]:
import pandas as pd

msft = pd.read_csv('./data/MSFT.csv')
msft.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-03-25,116.559998,118.010002,116.320000,117.660004,116.123627,27067100
1,2019-03-26,118.620003,118.709999,116.849998,117.910004,116.370346,26097700
2,2019-03-27,117.879997,118.209999,115.519997,116.769997,115.245224,22733400
3,2019-03-28,117.440002,117.580002,116.129997,116.930000,115.403145,18334800
4,2019-03-29,118.070000,118.320000,116.959999,117.940002,116.399963,25399800


In [2]:
import pandas as pd
from pyalgotrade import strategy
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.technical import ma, rsi, cross


class BuildFeatures(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        strategy.BacktestingStrategy.__init__(self, feed)
        self.__sma = ma.SMA(feed[instrument].getCloseDataSeries(), 15)
        self.__instrument = instrument
        self.cols = ["Date", "Close", "SMA_15"]
        self.features = pd.DataFrame(columns=self.cols)

    def onBars(self, bars):
        bar = bars[self.__instrument]
        # build up this list of signal features
        # right now it's just appending metrics but we can write methods to convert to signals
        technicals = pd.DataFrame([[
                bar.getDateTime(), 
                bar.getClose(),
                self.__sma[-1]
              ]], 
            columns=self.cols)
        self.features = self.features.append(technicals, ignore_index=True)
    
    def onFinish(self, bars):
        # to use this as a features set 
        # return self.featuers
        print(self.features.tail())
        

# Load the yahoo feed from the CSV file
feed = yahoofeed.Feed()
feed.addBarsFromCSV("MSFT", "./data/MSFT.csv")

# Evaluate the strategy with the feed's bars.
features = BuildFeatures(feed, "MSFT")
features.run()

          Date       Close      SMA_15
498 2020-03-18  140.399994  156.088666
499 2020-03-19  142.710007  155.057333
500 2020-03-20  137.350006  153.413334
501 2020-03-23  135.979996  150.959334
502 2020-03-24  148.339996  149.881334
